<a href="https://colab.research.google.com/github/oluwafemidiakhoa/MLprject/blob/main/Multi_Agent_System_fo_NASA_Mission_Planning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.3/389.3 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.0
    Uninstalling openai-1.54.0:
      Successfully uninstalled openai-1.54.0


In [ ]:
import os

os.environ['OPENAI_API_KEY'] =

In [ ]:
import os
import uuid
from openai import OpenAI

# Initialize the OpenAI client
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)


In [ ]:
# Base Agent Class
class Agent:
    def __init__(self, agent_id=None):
        self.agent_id = agent_id or str(uuid.uuid4())
        self.role = 'Agent'
        self.memory = []
        self.peers = []
        self.knowledge_base = None
        self.communication_channel = None

    def communicate(self, message, recipient_id=None):
        if self.communication_channel:
            self.communication_channel.send(self.agent_id, recipient_id, message)

    def receive_messages(self):
        if self.communication_channel:
            return self.communication_channel.receive(self.agent_id)
        return []

    def query_gpt(self, prompt):
        response = client.chat.completions.create(
            model='gpt-4',
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message.content


In [ ]:
# Data Analysis Agent
class DataAnalysisAgent(Agent):
    def __init__(self, agent_id=None):
        super().__init__(agent_id)
        self.role = 'Data Analysis Agent'

    def perform_task(self, data_description):
        prompt = f"""
You are a data analysis expert at NASA. Analyze the following dataset description and provide insights, trends, and potential areas of interest.

Dataset Description:
{data_description}
"""
        analysis = self.query_gpt(prompt)
        self.memory.append({'task': 'data_analysis', 'data': analysis})
        return analysis

# Anomaly Detection Agent
class AnomalyDetectionAgent(Agent):
    def __init__(self, agent_id=None):
        super().__init__(agent_id)
        self.role = 'Anomaly Detection Agent'

    def perform_task(self, data_description):
        prompt = f"""
You are an anomaly detection specialist at NASA. Examine the following data description for any anomalies or unusual patterns. Suggest possible causes and implications.

Data Description:
{data_description}
"""
        anomalies = self.query_gpt(prompt)
        self.memory.append({'task': 'anomaly_detection', 'data': anomalies})
        return anomalies

# Mission Planning Agent
class MissionPlanningAgent(Agent):
    def __init__(self, agent_id=None):
        super().__init__(agent_id)
        self.role = 'Mission Planning Agent'

    def perform_task(self, mission_objectives):
        prompt = f"""
You are a mission planner at NASA. Based on the following mission objectives, outline a detailed mission plan, including timelines, resource allocation, and potential risks.

Mission Objectives:
{mission_objectives}
"""
        plan = self.query_gpt(prompt)
        self.memory.append({'task': 'mission_planning', 'data': plan})
        return plan

# Knowledge Management Agent
class KnowledgeManagementAgent(Agent):
    def __init__(self, agent_id=None):
        super().__init__(agent_id)
        self.role = 'Knowledge Management Agent'

    def perform_task(self, query):
        prompt = f"""
You are a knowledge management specialist at NASA. Retrieve information relevant to the following query from the knowledge base.

Query:
{query}
"""
        information = self.query_gpt(prompt)
        self.memory.append({'task': 'knowledge_retrieval', 'data': information})
        return information

# Collaboration Agent
class CollaborationAgent(Agent):
    def __init__(self, agent_id=None):
        super().__init__(agent_id)
        self.role = 'Collaboration Agent'

    def perform_task(self, inputs):
        prompt = f"""
You are a collaboration facilitator at NASA. Integrate the following inputs from different agents into a cohesive report. Highlight key points and ensure consistency.

Inputs:
{inputs}
"""
        report = self.query_gpt(prompt)
        self.memory.append({'task': 'collaboration', 'data': report})
        return report

# Simulation Agent
class SimulationAgent(Agent):
    def __init__(self, agent_id=None):
        super().__init__(agent_id)
        self.role = 'Simulation Agent'

    def perform_task(self, scenario_description):
        prompt = f"""
You are a simulation expert at NASA. Based on the following scenario description, run a simulation and predict possible outcomes. Provide detailed results and recommendations.

Scenario Description:
{scenario_description}
"""
        simulation_results = self.query_gpt(prompt)
        self.memory.append({'task': 'simulation', 'data': simulation_results})
        return simulation_results


In [ ]:
# Communication Channel
class CommunicationChannel:
    def __init__(self):
        self.messages = []

    def send(self, sender_id, recipient_id, content):
        message = {'sender': sender_id, 'recipient': recipient_id, 'content': content}
        self.messages.append(message)

    def receive(self, agent_id):
        agent_messages = [
            msg for msg in self.messages
            if msg['recipient'] == agent_id or msg['recipient'] is None
        ]
        self.messages = [msg for msg in self.messages if msg not in agent_messages]
        return agent_messages


In [ ]:
# Knowledge Base (Simplified)
class KnowledgeBase:
    def __init__(self):
        self.data = {
            'missions': 'Information about past and current NASA missions.',
            'datasets': 'Descriptions of available datasets.',
            'technologies': 'Details about technologies used in space exploration.',
        }

    def store(self, key, value):
        self.data[key] = value

    def retrieve(self, key):
        return self.data.get(key, 'No information available.')


In [ ]:
# Initialize shared components
knowledge_base = KnowledgeBase()
communication_channel = CommunicationChannel()

# Create agents
data_agent = DataAnalysisAgent()
anomaly_agent = AnomalyDetectionAgent()
planning_agent = MissionPlanningAgent()
knowledge_agent = KnowledgeManagementAgent()
collaboration_agent = CollaborationAgent()
simulation_agent = SimulationAgent()

agents = [
    data_agent,
    anomaly_agent,
    planning_agent,
    knowledge_agent,
    collaboration_agent,
    simulation_agent,
]

# Assign shared components to agents
for agent in agents:
    agent.knowledge_base = knowledge_base
    agent.communication_channel = communication_channel
    agent.peers = [a.agent_id for a in agents if a.agent_id != agent.agent_id]


In [ ]:
# Define a mission scenario
mission_objectives = "Plan a lunar surface mission to study the South Pole-Aitken basin, focusing on geological sampling and water ice detection."

# Workflow Execution
def execute_workflow(objectives):
    # Step 1: Mission Planning
    print(f"\n{planning_agent.role} is creating a mission plan.")
    mission_plan = planning_agent.perform_task(objectives)
    planning_agent.communicate(mission_plan, simulation_agent.agent_id)

    # Step 2: Simulation
    print(f"\n{simulation_agent.role} is running simulations.")
    messages = simulation_agent.receive_messages()
    if messages:
        scenario_description = messages[0]['content']
        simulation_results = simulation_agent.perform_task(scenario_description)
        simulation_agent.communicate(simulation_results, data_agent.agent_id)

    # Step 3: Data Analysis
    print(f"\n{data_agent.role} is analyzing simulation data.")
    messages = data_agent.receive_messages()
    if messages:
        data_description = messages[0]['content']
        analysis = data_agent.perform_task(data_description)
        data_agent.communicate(analysis, anomaly_agent.agent_id)

    # Step 4: Anomaly Detection
    print(f"\n{anomaly_agent.role} is detecting anomalies.")
    messages = anomaly_agent.receive_messages()
    if messages:
        data_description = messages[0]['content']
        anomalies = anomaly_agent.perform_task(data_description)
        anomaly_agent.communicate(anomalies, knowledge_agent.agent_id)

    # Step 5: Knowledge Retrieval
    print(f"\n{knowledge_agent.role} is retrieving relevant information.")
    messages = knowledge_agent.receive_messages()
    if messages:
        query = f"Information related to anomalies detected: {messages[0]['content']}"
        knowledge = knowledge_agent.perform_task(query)
        knowledge_agent.communicate(knowledge, collaboration_agent.agent_id)

    # Step 6: Collaboration and Reporting
    print(f"\n{collaboration_agent.role} is compiling the final report.")
    inputs = []
    for agent in [planning_agent, simulation_agent, data_agent, anomaly_agent, knowledge_agent]:
        if agent.memory:
            inputs.append(f"{agent.role} Output:\n{agent.memory[-1]['data']}\n")
    compiled_inputs = "\n".join(inputs)
    final_report = collaboration_agent.perform_task(compiled_inputs)

    # Final Output
    print("\nFinal Mission Report:")
    print(final_report)

# Execute the workflow
execute_workflow(mission_objectives)




Mission Planning Agent is creating a mission plan.

Simulation Agent is running simulations.

Data Analysis Agent is analyzing simulation data.

Anomaly Detection Agent is detecting anomalies.

Knowledge Management Agent is retrieving relevant information.

Collaboration Agent is compiling the final report.

Final Mission Report:
Consolidated Report:
Our mission to the South Pole-Aitken basin comprises of different phases, each involving a distinct set of resources, activities, and potential risks. 

The first phase, Pre-Mission Preparation, will span over 1-2 years, and will involve consulting with earth- and space-based observational data, designing and developing the lunar rover, executing comprehensive training for the mission crew, running simulation tests, and establishing communication protocols. The risks associated with this phase primarily involve technical problems and training coverage.

The next phase, Launch and Lunar Transit, will last for an estimated 3-5 days. This ph